In [1]:
import pandas as pd
books = pd.read_csv("books_with_categories.csv")

Use dataloop.ai website to get more information on training and accuracy of these models for their previous runs.

In [3]:
from transformers import pipeline
classifier = pipeline("text-classification",
                      model="j-hartmann/emotion-english-distilroberta-base",
                      top_k=None,
                      device="mps")
classifier("I love this!")


Device set to use mps


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.00852868054062128},
  {'label': 'neutral', 'score': 0.00576459476724267},
  {'label': 'anger', 'score': 0.004419783595949411},
  {'label': 'sadness', 'score': 0.002092391485348344},
  {'label': 'disgust', 'score': 0.001611992483958602},
  {'label': 'fear', 'score': 0.00041385198710486293}]]

In [4]:
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

Here in this example, we can see, there are many feelings described. Lets see how the sentiment analyser works here

In [5]:
classifier(books["description"][0])

[[{'label': 'fear', 'score': 0.6548422574996948},
  {'label': 'neutral', 'score': 0.16985155642032623},
  {'label': 'sadness', 'score': 0.11640843003988266},
  {'label': 'surprise', 'score': 0.020700642839074135},
  {'label': 'disgust', 'score': 0.019100701436400414},
  {'label': 'joy', 'score': 0.015161211602389812},
  {'label': 'anger', 'score': 0.003935141488909721}]]

So we do not feel fear in this description as predicted by the model. So, lets break this description into individual sentences and then predict.

In [6]:
classifier(books["description"][0].split("."))


[[{'label': 'surprise', 'score': 0.7296021580696106},
  {'label': 'neutral', 'score': 0.14038625359535217},
  {'label': 'fear', 'score': 0.06816215068101883},
  {'label': 'joy', 'score': 0.0479423962533474},
  {'label': 'anger', 'score': 0.009156353771686554},
  {'label': 'disgust', 'score': 0.0026284793857485056},
  {'label': 'sadness', 'score': 0.0021221640054136515}],
 [{'label': 'neutral', 'score': 0.44937190413475037},
  {'label': 'disgust', 'score': 0.2735902965068817},
  {'label': 'joy', 'score': 0.1090828999876976},
  {'label': 'sadness', 'score': 0.09362740814685822},
  {'label': 'anger', 'score': 0.04047819972038269},
  {'label': 'surprise', 'score': 0.02697022259235382},
  {'label': 'fear', 'score': 0.00687906239181757}],
 [{'label': 'neutral', 'score': 0.6462160348892212},
  {'label': 'sadness', 'score': 0.24273350834846497},
  {'label': 'disgust', 'score': 0.043422576040029526},
  {'label': 'surprise', 'score': 0.02830047346651554},
  {'label': 'joy', 'score': 0.0142114348

In [7]:
sentences = books["description"][0].split(".")
predictions = classifier(sentences)

In [8]:
sentences[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [9]:
predictions[0]

[{'label': 'surprise', 'score': 0.7296021580696106},
 {'label': 'neutral', 'score': 0.14038625359535217},
 {'label': 'fear', 'score': 0.06816215068101883},
 {'label': 'joy', 'score': 0.0479423962533474},
 {'label': 'anger', 'score': 0.009156353771686554},
 {'label': 'disgust', 'score': 0.0026284793857485056},
 {'label': 'sadness', 'score': 0.0021221640054136515}]

In [10]:
sentences[3]

' Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist'

In [11]:
predictions[3]

[{'label': 'fear', 'score': 0.9281686544418335},
 {'label': 'anger', 'score': 0.032190557569265366},
 {'label': 'neutral', 'score': 0.012808652594685555},
 {'label': 'sadness', 'score': 0.00875683594495058},
 {'label': 'surprise', 'score': 0.008597896434366703},
 {'label': 'disgust', 'score': 0.008431782014667988},
 {'label': 'joy', 'score': 0.0010455816518515348}]

The idea here is to take max of each label from the whole description. First the output is in different order. so we wil sort that to maintain a order for the rest of the processing.

In [12]:
sorted(predictions[0],key=lambda x:x["label"])

[{'label': 'anger', 'score': 0.009156353771686554},
 {'label': 'disgust', 'score': 0.0026284793857485056},
 {'label': 'fear', 'score': 0.06816215068101883},
 {'label': 'joy', 'score': 0.0479423962533474},
 {'label': 'neutral', 'score': 0.14038625359535217},
 {'label': 'sadness', 'score': 0.0021221640054136515},
 {'label': 'surprise', 'score': 0.7296021580696106}]

In [14]:
import numpy as np

emotion_labels = ['anger','disgust','fear','joy','neutral','sadness','surprise']
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x:x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}


In [16]:
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

In [17]:
emotion_scores

{'anger': [np.float64(0.06413352489471436),
  np.float64(0.6126183867454529),
  np.float64(0.06413352489471436),
  np.float64(0.35148337483406067),
  np.float64(0.08141220360994339),
  np.float64(0.23222479224205017),
  np.float64(0.5381850600242615),
  np.float64(0.06413352489471436),
  np.float64(0.3006710410118103),
  np.float64(0.06413352489471436)],
 'disgust': [np.float64(0.2735902965068817),
  np.float64(0.34828585386276245),
  np.float64(0.1040065661072731),
  np.float64(0.15072239935398102),
  np.float64(0.1844952553510666),
  np.float64(0.7271748781204224),
  np.float64(0.15585508942604065),
  np.float64(0.1040065661072731),
  np.float64(0.279480904340744),
  np.float64(0.177926704287529)],
 'fear': [np.float64(0.9281686544418335),
  np.float64(0.9425276517868042),
  np.float64(0.9723208546638489),
  np.float64(0.3607071340084076),
  np.float64(0.09504334628582001),
  np.float64(0.05136272683739662),
  np.float64(0.7474281787872314),
  np.float64(0.404495507478714),
  np.floa

In [18]:
from tqdm import tqdm
#reset
emotion_labels = ['anger','disgust','fear','joy','neutral','sadness','surprise']
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 5197/5197 [05:33<00:00, 15.58it/s]


In [19]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbn

In [20]:
emotions_df

,anger,disgust,fear,joy,neutral,sadness,surprise,isbn13
0,0.064134,0.273590,0.928169,0.932798,0.646216,0.967158,0.729602,9780002005883
1,0.612618,0.348286,0.942528,0.704422,0.887940,0.111690,0.252544,9780002261982
2,0.064134,0.104007,0.972321,0.767238,0.549477,0.111690,0.078765,9780006178736
3,0.351483,0.150722,0.360707,0.251881,0.732687,0.111690,0.078765,9780006280897
4,0.081412,0.184495,0.095043,0.040564,0.884389,0.475881,0.078765,9780006280934
...,...,...,...,...,...,...,...,...
5192,0.148208,0.030643,0.919165,0.255169,0.853722,0.980877,0.030656,9788172235222
5193,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765,9788173031014
5194,0.009997,0.009929,0.339218,0.947779,0.375755,0.066685,0.057625,9788179921623
5195,0.064134,0.104007,0.459270,0.759456,0.951104,0.368110,0.078765,9788185300535


In [21]:
#merge to original DF
books = pd.merge(books, emotions_df, on="isbn13")

In [22]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,neutral,sadness,surprise
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.064134,0.273590,0.928169,0.932798,0.646216,0.967158,0.729602
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.612618,0.348286,0.942528,0.704422,0.887940,0.111690,0.252544
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.064134,0.104007,0.972321,0.767238,0.549477,0.111690,0.078765
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.351483,0.150722,0.360707,0.251881,0.732687,0.111690,0.078765
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,0.081412,0.184495,0.095043,0.040564,0.884389,0.475881,0.078765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Fiction,0.148208,0.030643,0.919165,0.255169,0.853722,0.980877,0.030656
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction,0.009997,0.009929,0.339218,0.947779,0.375755,0.066685,0.057625
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction,0.064134,0.104007,0.459270,0.759456,0.951104,0.368110,0.078765


In [23]:
#save to file
books.to_csv("books_with_emotions.csv", index=False)